In [26]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
%matplotlib qt

import phase

In [27]:
data = pd.read_pickle('T:/Team/Szewczyk/Data/20240109/data.pkl')

In [13]:
plt.plot(data['t'], data['I'])

In [29]:
pert_times = np.array(data[np.diff(data['U'], prepend=4) > 0.1]['t'])
time_roi = (pert_times[0] - 80, pert_times[-1] - 80)

data = data[(data['t'] > time_roi[0]) & (data['t'] < time_roi[1])]

In [30]:
def find_cycles(data, pert_times, threshold_I_multiplier=1.05, sign_change = -1):
    '''
    Divide the signal into cycles by cutting when the current
    crosses a specific value.

    Parameters
    ----------
    data : pd.DataFrame
        experimental data
    threshold_I_multiplier : float
        point where I == threshold_I is defined
        to have phase = 0; defined as fraction of mean current
    sign_change : int
        1 when I raises at phase==0, otherwise -1

    
    Returns
    -------
    cycles : pd.DataFrame
        A dataframe describing period information
            start -- t at which phase == 0
            duration -- T of this period
    '''
    
    # Calculate current relative to the threshold
    threshold_I = data.mean()['I']*threshold_I_multiplier
    data['I relative'] = data['I']-threshold_I

    # Calculate crossings and create 'cycles' dataframe
    crossings = data[(np.diff(np.sign(data['I relative']), append=0) == 2*sign_change)]
    crossing_times = np.array(crossings['t'])
    period_durations = np.diff(crossing_times)
    voltage = np.array(crossings['U'])


    dt = np.min(np.abs(np.subtract.outer(crossing_times, pert_times)), axis=1)



    unpurged_cycles = pd.DataFrame({'start'     : crossing_times,
                                    'duration'  : np.append(period_durations, np.nan),
                                    'U'         : voltage,
                                    'dt'        : dt})
    
    # Remove false crossings (when period is too short)
    # mean_period = np.mean(unpurged_cycles['duration'])
    # cycles = unpurged_cycles[(unpurged_cycles['duration'] > 40)]

    cycles = unpurged_cycles[(unpurged_cycles['U'] < 4.05)]
    cycles = cycles[(cycles['dt'] > 1)]  
    cycles = cycles.assign(duration= np.diff(cycles['start'], append=np.nan))
    cycles = cycles[(cycles['duration'] > 40) & (cycles['duration'] < 70)] 

    cycles = cycles.drop(cycles.tail(1).index) # Drop last row
    cycles.reset_index(drop=True, inplace=True)
    
    return unpurged_cycles, threshold_I

In [31]:
cycles, threshold_I = find_cycles(data, pert_times, threshold_I_multiplier=1.05, sign_change=1)
plt.plot(cycles['duration'])